# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1.

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1]:
# импортируем необходимые библиотеки
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time

In [2]:
# response 200 говорит о том, что все работает корректно 
result = requests.get('https://habr.com/ru/all/')
result

<Response [200]>

In [3]:
# ключевые слова для поиска и ссылка
KEYWORDS = ['python', 'парсинг']
link_ = 'https://habr.com/ru/all/'

In [4]:
# получаем страницу с самыми свежими постами
result = requests.get(link_)
soup = BeautifulSoup(result.text, 'html.parser')

In [5]:
# получаем все статьи
posts = soup.find_all('article', class_='post')
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/oldadmin/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/7d0/232/a49/7d0232a498a452229600e7d49a1391ef.png" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">oldadmin</span>
 </a>
 <span class="post__time">сегодня в 16:10</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/ruvds/blog/532140/">Wireguard для Kubernetes и удобные GUI</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/company/ruvds/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Блог компании RUVDS.com'); }" rel="nofollow" title="Вы не подписаны на этот

In [6]:
# получаем ссылки и заголовки статей 
post_head = list(map(lambda x: x.find('h2', class_='post__title'), posts))
post_head

[<h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/ruvds/blog/532140/">Wireguard для Kubernetes и удобные GUI</a>
 </h2>,
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/microsoft/blog/532148/">Гибридные облачные технологии, как основа для успешной цифровой трансформации</a>
 </h2>,
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/southbridge/blog/532134/">Kubernetes 1.20 — и что же слома.../починили на этот раз?</a>
 </h2>,
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/amnezia/blog/531720/">Amnezia: всё будет забыто</a>
 </h2>,
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/otus/blog/532078/">Семь талантливых стажеров AI@Unity 2020. Часть 2</a>
 </h2>,
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/tuturu/blog/532136/">МКПП: что это за система

In [7]:
# добираемся до ссылок
post_link = list(map(lambda x: x.find('a').get('href'), post_head))
post_link

['https://habr.com/ru/company/ruvds/blog/532140/',
 'https://habr.com/ru/company/microsoft/blog/532148/',
 'https://habr.com/ru/company/southbridge/blog/532134/',
 'https://habr.com/ru/company/amnezia/blog/531720/',
 'https://habr.com/ru/company/otus/blog/532078/',
 'https://habr.com/ru/company/tuturu/blog/532136/',
 'https://habr.com/ru/post/532132/',
 'https://habr.com/ru/company/flipperdevices/blog/532028/',
 'https://habr.com/ru/post/532126/',
 'https://habr.com/ru/company/selectel/blog/532122/',
 'https://habr.com/ru/company/otus/blog/532056/',
 'https://habr.com/ru/post/532112/',
 'https://habr.com/ru/post/532114/',
 'https://habr.com/ru/company/madrobots/blog/531980/',
 'https://habr.com/ru/post/532104/',
 'https://habr.com/ru/post/532100/',
 'https://habr.com/ru/post/532098/',
 'https://habr.com/ru/post/532094/',
 'https://habr.com/ru/company/ruvds/blog/529840/']

In [8]:
# собираем даты, заголовки и тексты новостей и тексты всех статей 
for link in post_link:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)
    date = soup.find('span', class_='post__time').get('data-time_published')
    print(date)
    title = soup.find('span', class_='post__title-text').text
    print(title)
    text = soup.find('div', class_='post__text').text
    print(text)

2020-12-09T13:10Z
Wireguard для Kubernetes и удобные GUI

Can I just once again state my love for [WireGuard] and hope it gets merged soon? Maybe the code isn't perfect, but I've skimmed it, and compared to the horrors that are OpenVPN and IPSec, it's a work of art.

Linus Torvalds, on the Linux Kernel Mailing List
Уверен, что многие уже слышали про Wireguard. Некоторые даже щупали вживую. Мне он вполне понравился в продакшене как по производительности, так и по простоте настройки. Да, там нет миллиона загадочных конфигураций на все случаи жизни как у OpenVPN и XFRM+StrongSwan в IPSEC VPN. Он прост и лаконичен как в коде, так и в конфигах. Жирным и неповоротливым он станет еще нескоро. Короче, такой позитивный вариант сценария «сжечь легаси и написать все как надо».

Я решил не писать очередной гайд «как я поставил Wireguard», а поделиться некоторыми полезными утилитами. Некоторые с очень удачным, на мой вкус GUI, что уместно для сервисов со множеством пользователей. Разберем использов

2020-12-09T12:54Z
Kubernetes 1.20 — и что же слома.../починили на этот раз?
Release Logo c сайта kubernetes.ioПоздравляем с выходом версии 1.20. Третий релиз в 2020 году, в котором 11 фич объявили stable, 15 перевели в beta и добавили 16 новых в alpha-стадии.Судя по Release Logo, котики уже захватили мир и даже не скрывают этого. Давайте посмотрим, какие блюдца с молоком они заботливо положили нам под ноги.Конечно же, начать придется с громкой новости об отказе от dockershim. С одной стороны, дело это не быстрое, и есть еще целый год перед окончательным выпиливанием кода, но, с другой стороны, можно ожидать, что баги при взаимодействии с docker будут исправляться с меньшим приоритетом, чем раньше.Второе важное изменение, которое может сломать ваш продакшен, заключается в том, что в версии 1.20 починили баг с таймаутами для Probe типа exec.В описании контейнеров в манифесте подов есть специальный раздел, описывающий каким образом можно узнать, что приложение живое и готово обрабатывать 

2020-12-09T12:28Z
Семь талантливых стажеров AI@Unity 2020. Часть 2
Будущих студентов курса "Unity Game Developer. Basic" приглашаем принять участие в открытом уроке на тему "2d-платформер головоломка".А также делимся продолжением перевода интересного материала.PSankalp Patro (ML-Agents): Отслеживание выходных данных моделиОбучение с подкреплением требует значительных вычислительных ресурсов, которые зачастую намного больше, чем среднестатистический пользователь может аккумулировать на своем ноутбуке или персональном компьютере. Чтобы упростить нашим пользователям обучение, команда ML-Agents в настоящее время разрабатывает ML-Agents Cloud — экспериментальный обучающий облачный сервис для ML-Agents (заявленный в нашем блоге Release 1).ML-Agents Cloud предлагает пользователям ряд преимуществ. Теперь разработчики могут проводить несколько исследований параллельно, а не быть ограниченными одним исследованием за раз на своем локальном компьютере. Благодаря передовым вычислительным мощностям 

2020-12-09T12:10Z
МКПП: что это за система и почему нелегальных автобусных перевозок станет сильно меньше
С декабря 2020 года автобусный билет будет снабжаться метаданными, которые можно будет проверить через МКПП — это единая система, действующая по всей стране. То есть пассажир будет уверен в перевозчике, маршруте и его официальности, на старте, на междугороднем маршруте.

«Палёные» автобусы — это бич транспортной системы нашей страны. Речь про широкий спектр от «Ларгусов» и ПАЗиков, которые маскируются под частные перевозки на сервисах поиска попутчиков, до просто целых больших автобусов, водители которых никому ничего не должны. Мы слышали всякое: и высадку пассажиров прямо в поле, и когда водитель отказался везти двух человек, потому что не хотел ехать до конечного пункта, и чудесное превращение в кабриолет из-за ржавой крыши на ходу. 

Официальный рейс от неофициального отличается несколькими важными вещами: гарантиями пассажирам, обязательностью исполнения, медосмотром водителя,

2020-12-09T12:08Z
DevOps-дайджест от «Рексофт»
Привет, Хабр!Предлагаем подборку свежих статей, подкастов, докладов и грядущих событий для DevOps-специалистов от команды «Рексофт». Собрали всё самое интересное за последние несколько недель.Статьи1. Обзор инструментов для chaos engineering в KubernetesМатериал в двух частях от специалистов из компании «Флант». Они описывают и сравнивают Open Source-утилиты для запуска «управляемого хаоса» в кластере Kubernetes. В первой части специалисты рассказали о появлении самой дисциплины chaos engineering, а также рассмотрели kube-monkey, chaoskube и Chaos Mesh.Читать первую частьВо второй статье список пополнится обзором Litmus Chaos, Chaos Toolkit, мини-подборкой из хаос-игр и пятью другими вариантами, заслуживающих внимания инженеров, заинтересованных в проверке инфраструктуры на устойчивость.Читать вторую часть2. Не паникуйте: Kubernetes и DockerНачиная с версии v1.20, Kubernetes отказывается от Docker как от исполняемой среды контейнеров. Но н

2020-12-09T11:28Z
PuLP-MiA: Мультииндексный аддон для PuLP (python-библиотека линейного программирования)
Привет, Хабр! Сейчас будет мини-пост без единой строки кода для тех, кто имеет дело с многоиндексными задачами ЛП (линейное программирование) в Python и решает их при помощи библиотеки-порта PuLP… Это ненадолго :-)

При формализации задач ЛП довольно часто приходится иметь дело с многоиндексными переменными. При работе с задачами большой размерности — это, прямо скажем, привычное дело.

Взаимосвязи таких многоиндексных переменных в целевой функции (линейная форма — она же линейный критерий оптимизации) и ограничениях (в виде линейных равенств и неравенств) следует генерировать программно. При работе с PuLP (библотека-порт ЛП для Python) используют два основных подхода к такой генерации:

Генерация матрицы А (матрица ограничений) с помощью генераторов списков Python явным образом. Например, так: Sudoku problem
Генерация символьных переменных с привязкой к индексам через словари в не

2020-12-09T10:40Z
Невидимый гость: вскрываем беспроводную сеть компании с Kali Linux и Raspberry Pi


Сегодня рассмотрим, как относительно незаметно протестировать безопасность беспроводной сети компании. В качестве основы будет использоваться Raspberry Pi, который поддерживает установку Kali Linux. Установка дистрибутива довольно проста:


Загрузить официальный образ с сайта kali.org;
Записать его на SD-диск с помощью Win32image для Windows и Gparted для Linux;
Запустить Raspberry Pi с установленной в него SD-картой.

После установки, по желанию, можно обновить пакеты, если они будут доступны. Но для полноценной работы больше не требуется никаких действий. Требуемый размер карты памяти — 8Гб и выше. Чтобы система нормально функционировала, желательно использовать больший объем.

Статья носит информационный характер. Не нарушайте законодательство.
Теперь, когда система готова к работе, осталось только настроить запуск программ при старте системы, т.к. предполагается, что Raspberry Pi б

2020-12-09T10:30Z
Коротко об одном из стандартов формата токена PKCS#15
Много статей написано про стандарты API между токенами и программным обеспечением. Однако, стандартизация формата хранения данных на самих токенах, как правило, несправедливо обойдена стороной. В данной статье кратко описан один из стандартов формата токена PKCS#15 (Public Key Cryptography standard), опубликованного RSA Laboratories.Криптографический токен и зачем стандартизировать формат его храненияКриптографический токен - портативное устройство, способное хранить криптографические учетные данные, идентифицирующие его владельца (например, смарт-карты).  Они отлично подходят для обеспечения безопасности и приватности в приложениях. На них может храниться в защищенном формате важная информация, такая как пароли, секретные ключи,  закрытые ключи и т.д.Токены предоставляют изолированное пространство, в котором можно обрабатывать чувствительную информацию, чтобы не открывать ее в самих приложениях, подверженных разли

2020-12-09T10:01Z
Не работает поиск в Windows 10
В Windows 10 имеется удобная функция, которой я постоянно пользуюсь - поиск в меню "Пуск". Конкретно имею ввиду возможность нажать на клавиатуре клавишу "Пуск" и сразу начать вводить, например, первые буквы названия нужной программы - результаты поиска сразу отображаются на экране.В какой-то момент эта функция перестала работать на моём рабочем компьютере. Незадолго до этого возможность поиска в меню "Пуск" пропала и у моего коллеги.Также я обнаружил, что не работает поиск в панели задач (если включено отображение кнопки поиска): при вводе текста в панель поиска либо отображается просто чёрное окно, либо - вообще ничего.Но и это ещё не всё! В панели задач для отображаемых значков перестало выводиться контекстное меню по нажатию правой кнопки мыши.Долгие поиски на русскоязычных и англоязычных сайтах приводили примерно к одним и тем же результатам: глюки сервисов Microsoft в марте 2020, переиндексация, откат обновлений и прочее. Но результ

2020-12-09T09:47Z
Кунг-фу стиля Linux: утилита marker и меню для командной строки
Командная строка… Её можно любить или ненавидеть, но тому, кто пользуется Unix-подобными операционными системами, без неё не обойтись. Облегчить работу в командной строке можно с помощью утилиты marker. В её репозитории о ней говорят как о «панели команд для терминала». Полагаю, это — довольно полезная программа. Не могу подобрать точное сравнение для её описания. Не знаю, похожа ли она на продвинутую историю команд, или на систему управления закладками. Пожалуй, в ней есть немного и того и другого.



История работы с командами теряется, она содержит множество мелких команд (хотя, чтобы в историю не попадали бы определённые команды, можно воспользоваться переменной окружения HISTIGNORE). А при использовании marker можно сохранить определённые команды, после чего они уже не потеряются. В хранилище marker не попадает ничего лишнего, а то, что было сохранено, никуда не девается.

Конечно, можно просто напис

In [9]:
result_df = pd.DataFrame()
for link in post_link:
    linkget = requests.get(link)
    time.sleep(0.3)
    soup = BeautifulSoup(linkget.text, 'html.parser')
    # получаем полный текст статьи
    post_full_text = soup.find('div', class_='post__body post__body_full').text
    # проверяем на наличие ключевых слов
    if any([keyword.lower() in post_full_text.lower() for keyword in KEYWORDS]):
        # находим дату поста, заголовок и полный текст
        post_date = soup.find('span', class_='post__time').get('data-time_published')
        post_title = soup.find('span', class_='post__title-text').text
        res_string = {'post_date': post_date, 'post_title': post_title, 'URL': link, 'Full_Text': post_full_text}
        result_df = pd.concat([result_df, pd.DataFrame([res_string])])
result_df

,post_date,post_title,URL,Full_Text
0,2020-12-09T11:34Z,Помогите написать лор для Flipper Zero,https://habr.com/ru/company/flipperdevices/blo...,\n\n\r\nЛор (от англ. Lore) — это история всел...
0,2020-12-09T11:28Z,PuLP-MiA: Мультииндексный аддон для PuLP (pyth...,https://habr.com/ru/post/532126/,"\nПривет, Хабр! Сейчас будет мини-пост без еди..."


## Задание 2

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [10]:
res = requests.post('https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data')
res

<Response [400]>

In [24]:
mail_df = pd.DataFrame()
EMAIL = ['noob28@mail.ru']
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}
addresses = {
    'emailAddresses': EMAIL, 
    'Data': 'Thu, 03 Dec 2020 12:48:39 GMT'
}
res = requests.post(url, json=addresses, headers=headers)
res_df = {'emailAddresses': addresses['emailAddresses'], 'Data': data['Data'], }
mail_df = pd.concat([mail_df, pd.DataFrame([res_df])])
mail_df


,emailAddresses,Data
0,[noob28@mail.ru],"Thu, 03 Dec 2020 12:48:39 GMT"


In [40]:
from pprint import pprint
pprint(res.json())

{'breaches': {'12': {'breachId': 12,
                     'description': 'Popular Russian social networking '
                                    'platform VKontakte was breached in late '
                                    '2012. Over 100 million clear-text '
                                    'passwords were compromised in the breach. '
                                    'Breached credential sets included '
                                    "victims' e-mail addresses, passwords, "
                                    'dates of birth, phone numbers and '
                                    'location details. The credential set was '
                                    'advertised on a dark web marketplace as '
                                    'of June 2016 for a price of one bitcoin. ',
                     'publishDate': '2016-10-29T00:00:00Z',
                     'recordsCount': 91262655,
                     'site': 'vk.com',
                     'statistics': {'emails': 91

In [47]:
result = res.json()

In [68]:
for key, val in result['breaches'].items():
    res_string = {'breach_id': val['breachId'], 'breach_date': val['publishDate'], 'site': val['site'], 'description': val['description']}  

In [52]:
res_string

{'breach_id': 17830,
 'breach_date': '2020-07-23T00:00:00Z',
 'site': 'wattpad.com',
 'description': 'In June 2020, the online writing community Wattpad was allegedly breached. The stolen data contains usernames, IPs, passwords, email addresses and additional personal information. This breach is being privately shared on the internet.',
 'mail': 'noob28@mail.ru'}

In [82]:
DATA = {'email': res_string['mail'], 'leak_date': res_string['breach_date'], 'leak_source': res_string['site'], 'description': res_string['description']}
leaks_df = pd.DataFrame(DATA)

KeyError: 'mail'

In [80]:
DATA

{'email': [],
 'leak_date': '2018-03-20T00:00:00Z',
 'leak_source': 'myfitnesspal.com',
 'description': "In February 2018, MyFitnessPal's database was breached. The stolen data contains usernames, IPs, passwords, salts, email addresses and additional personal information. The data has been sold on at least one dark web market and is also being privately shared amongst several criminal groups."}

In [81]:
leaks_df

,email,leak_date,leak_source,description


In [84]:
result['data']

{'eyeem.com': {'noob28@mail.ru': [{'breachId': 16401,
    'usernameBreached': True,
    'passwordBreached': True}]},
 'myfitnesspal.com': {'noob28@mail.ru': [{'breachId': 13916,
    'usernameBreached': True,
    'passwordBreached': True}]},
 'vk.com': {'noob28@mail.ru': [{'breachId': 12,
    'usernameBreached': True,
    'passwordBreached': True}]},
 'wattpad.com': {'noob28@mail.ru': [{'breachId': 17830,
    'usernameBreached': True,
    'passwordBreached': True}]}}

In [83]:
breach_data = pd.DataFrame()
for key, value in result['breaches'].items():
    res_string = {'breach_id': value['breachId'], 'breach_date': value['publishDate'], 'site': value['site'], 'description': value['description']}
    add_mail = ','.join(list(result['data'][res_string['site']]))
    res_string.update({'mail':add_mail})
    breach_data = pd.concat([breach_data, pd.DataFrame([res_string])])
breach_data = breach_data.reset_index().drop(['index'],axis=1)
breach_data

,breach_id,breach_date,site,description,mail
0,17830,2020-07-23T00:00:00Z,wattpad.com,"In June 2020, the online writing community Wat...",noob28@mail.ru
1,16401,2018-12-13T00:00:00Z,eyeem.com,"In February of 2018, the photography marketpla...",noob28@mail.ru
2,12,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...,noob28@mail.ru
3,13916,2018-03-20T00:00:00Z,myfitnesspal.com,"In February 2018, MyFitnessPal's database was ...",noob28@mail.ru
